In [3]:
import sys

sys.path.append("/Users/j/Documents/Forme/code/forme-groups")

from src.groups.unit import Unit, UnitType, UnitTypeRef, UnitTypeFunction
from src.groups.group import Group, Data


def main():
    # Create a default group
    group = Group(
        system_types_path="/Users/j/Documents/Forme/code/forme-groups/src/groups/system_types.json"
    )

    # Create function to be used as a system function
    def test_function(input_str):
        if input_str.startswith("test<") and input_str.endswith(">") and "|" in input_str:
            return input_str[5:-1].split("|")
        elif input_str.startswith("test<") and input_str.endswith(">"):
            return input_str[5:-1]
        else:
            return input_str

    # Create a new unit type
    new_unit_type = UnitType(
        aliases= (UnitTypeRef("test_unit_type"),),
        super_type=UnitTypeRef("str"),
        prefix="test<",
        suffix=">",
        separator="|",
        sys_function=UnitTypeFunction(
            object=test_function,
            args=(),
        )
    )

    # Add a unit type to the group
    group._group_unit_generator.unit_generator.unit_type_pool.add_unit_type(
        new_unit_type
    )

    # Freeze the unit types
    group.freeze_unit_types()
    print(group._group_unit_generator.unit_generator.unit_type_pool.get_type_aliases())
    print(group.group_units)
    
    # Create a new unit - Nonce: 0
    unit1 = group.create_group_unit()
    print(unit1)

    # Create another new unit - Nonce: 1
    group.create_group_unit()
    print(group.get_nonce_tiers())
    print(group.group_units)

    # Create a new unit with data - Nonce: 2
    data_unit: Unit = group._group_unit_generator.unit_generator.create_unit(
        alias="test_unit_type",
        value="test<test1|test2|test3>"
    )
    data_object: Data = group._group_unit_generator.create_data(data=(data_unit,))
    new_unit = group.create_group_unit(data=data_object)
    print(new_unit)
    print(group._group_unit_generator.unit_generator.format_unit(new_unit.data.entries[0]))
    print(group.format_group_unit(new_unit))
    #print(group.format_group_unit())

    # Create a new unit with a schema - Nonce: 3
    schema_unit: Unit = group._group_unit_generator.unit_generator.create_unit(
        alias="dict",
        value={'Schema': {'test': 'str'}}
    )
    schema_object: Data = group._group_unit_generator.create_data(data=(schema_unit,))
    new_unit = group.create_group_unit(data=schema_object)
    print(new_unit)
    print(new_unit.data.get_schema())
    print(group.group_units)

    print(group.get_all_group_units()[0].nonce)
    print(group.__slots__)
    print(hash(group))


main()

['string', 'str', 'int', 'integer', 'float', 'bool', 'boolean', 'list', 'dict', 'dictionary', 'tuple', 'bytes', 'schema', 'Schema', 'test_unit_type']
[GroupUnit(nonce=None, ownership=None, credentials=None, data=None)]
Creating new group unit with default nonce.


ValueError: Nonce already exists in GroupUnit.

In [1]:
import sys

sys.path.append("/Users/j/Documents/Forme/code/forme-groups")

from src.groups.unit import Unit, UnitType, UnitTypeRef, UnitTypeFunction, UnitGenerator
# from src.groups.nonce import Nonce
from src.groups.pool import Pool, UnitTypePool


def main():

    generic_pool = Pool(items=["test0"])
    generic_pool.add("test1")
    generic_pool.add("test2")
    generic_pool.add("test4")

    print(generic_pool)


    type_str = UnitType(aliases=[UnitTypeRef("test")], super_type=[UnitTypeRef("test")], prefix="", suffix="", separator="", sys_function=UnitTypeFunction(object=str, args=()))
    # type_pool.add(type_str)
    type_pool = UnitTypePool(items=(type_str,))
    # hash1 = hash(type_pool)
    type_pool.set_types_from_json("/Users/j/Documents/Forme/code/forme-groups/src/groups/system_types.json")
    type_pool.freeze()
    print(type_pool.items[0].__repr__())
    print(type_pool.items[0].hash_256())
    print(type_pool.items[1].hash_256())

    # print(item for item in type_pool)
    print(type_pool.get_type_aliases())
    print(type_str.__slots__)
    print(type_pool.__slots__)


    unit_type_ref = UnitTypeRef("test")
    unit_type_ref2 = UnitTypeRef("test")
    print(unit_type_ref.hash_256())
    unit_type_ref_hash = hash(unit_type_ref)
    print(unit_type_ref_hash)

    unit_type_hash2 = unit_type_ref.hash_256()
    print(unit_type_hash2)
    print(unit_type_ref2.hash_256())
    
    unit_type_funciton = UnitTypeFunction(object=str, args=())
    unit_type_funciton2 = UnitTypeFunction(object=str, args=())
    print(unit_type_funciton.hash_256())
    print(unit_type_funciton2.hash_256())





main()

Pool(items=['test0', 'test1', 'test2', 'test4'])
c712397ed1ec5bb29f878e4703ea2d0867739e2ad2f51a11349875f84f13737b
234386dbbc3fcd501021a7fe5a880efef74aaedabf2c6cc60e4913303ce20198
['test', 'string', 'str', 'int', 'integer', 'float', 'bool', 'boolean', 'list', 'dict', 'dictionary', 'tuple', 'bytes', 'schema', 'Schema']
('aliases', 'super_type', 'prefix', 'suffix', 'separator', 'sys_function', '__weakref__')
('_frozen', 'items')
a1a63976d57e9c546c11b08e64be6bb8a62aca996d438d1cd64ec6b61104e06b
2778305579550660780
a1a63976d57e9c546c11b08e64be6bb8a62aca996d438d1cd64ec6b61104e06b
a1a63976d57e9c546c11b08e64be6bb8a62aca996d438d1cd64ec6b61104e06b
1b68083e4c87fb1c18d2212f545129326c6f72647090aeb1b54f3d65a4279794
1b68083e4c87fb1c18d2212f545129326c6f72647090aeb1b54f3d65a4279794
